# Mission to Mars

Prep notebook

In [1]:
# Modules
import pandas as pd
import requests
from splinter import Browser
from bs4 import BeautifulSoup as bs

In [2]:
#ChromeDriver 
executable_path = {'executable_path': 'chromedriver'}
browser = Browser('chrome', **executable_path, headless = False )

Scrape news site and pull Mars specific articles.

In [4]:
#Connect to NASA Mars news site
nasaUrl = 'https://mars.nasa.gov/news'
browser.visit(nasaUrl)

#Create the HTML object
html = browser.html

#Time for BeautifulSoup
soup = bs(html, 'html.parser')

#Look up latest news title and paragraph
newsTitle = soup.find('div', class_ = 'content_title').find('a').text
newsParagraph = soup.find('div', class_ = 'article_teaser_body').text

#See if it worked
print(newsTitle)
print(newsParagraph)

Mars 2020 Unwrapped and Ready for More Testing
In time-lapse video, bunny-suited engineers remove the inner layer of protective foil on NASA's Mars 2020 rover after it was relocated for testing.


Scrape for Mars images.

In [5]:
#JPL image site
pixUrl = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(pixUrl)

#Create the html object
htmlImage = browser.html

#Parse with BS
soup = bs(htmlImage, 'html.parser')

#Featured image
featuredImage = soup.find('article')['style'].replace('background-image: url(','').replace(');','')[1:-1]

#Main url
mainUrl = 'https://www.jpl.nasa.gov'

#Create link
featuredImage = mainUrl + featuredImage

print(featuredImage)


https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA18289-1920x1200.jpg


Scrape twitter for mars weather update.

In [6]:
#Mars weather twitter
marsTwitter = 'https://twitter.com/marswxreport?lang=en'
browser.visit(marsTwitter)

#Html object
htmlMars = browser.html

#Noodle time
soup = bs(htmlMars, 'html.parser')

#Scrape it
marsTweets = soup.find_all('div', class_ = 'js-tweet-text-container')

#print(marsTweets)

#Find related tweets
for tweet in marsTweets:
    marsWeather = tweet.find('p').text
    if "sol" and "pressure" in marsWeather:
        print(f'Mars Weather: \n{marsWeather}')
        break
    else:
        pass

Mars Weather: 
InSight sol 316 (2019-10-16) low -101.8ºC (-151.3ºF) high -25.7ºC (-14.3ºF)
winds from the SSE at 4.7 m/s (10.5 mph) gusting to 18.2 m/s (40.8 mph)
pressure at 7.10 hPapic.twitter.com/tXtGZA6IPW


Scrape Mars facts.

In [7]:
#Mars facts url
marsFacts = 'https://space-facts.com/mars'

#Scrape for Mars facts
marsFactsDb = pd.read_html(marsFacts)[0]

#print(marsFactsDb)

#Pretty up
marsFactsDb.columns = ['Description', 'Mars Value', 'Earth Values']
marsFactsDb.set_index('Description', inplace = True)

print(marsFactsDb)

                         Mars Value     Earth Values
Description                                         
Diameter:                  6,779 km        12,742 km
Mass:               6.39 × 10^23 kg  5.97 × 10^24 kg
Moons:                            2                1
Distance from Sun:   227,943,824 km   149,598,262 km
Length of Year:      687 Earth days      365.24 days
Temperature:          -153 to 20 °C      -88 to 58°C


In [8]:
#Convert data to html table
marsFactsDb.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars Value</th>\n      <th>Earth Values</th>\n    </tr>\n    <tr>\n      <th>Description</th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>Distance from Sun:</th>\n      <td>227,943,824 km</td>\n      <td>149,598,262 km</td>\n    </tr>\n    <tr>\n      <th>Length of Year:</th>\n      <td>687 Earth days</td>\n      <td>365.24 days</td>\n    </tr>\n    <tr>\n      <th>Temperature:</th>\n      <td>-153 to 20 °C</td>\n      <td>-88 to 58°C</td>\n    </tr>\n  </tbody>\n</table>'

Scrape Hemisphere info

In [9]:
#Hemisphere Url
marsHemi = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(marsHemi)

#One more HTML object
htmlHemi = browser.html

#Sopita
soup = bs(htmlHemi, 'html.parser')

#Pull items that contain hemi info
marsItems = soup.find_all('div', class_ = 'item')

#Create placeholders
hemiImages = []
#hemiTitles = []

#Obtain the images
for i in marsItems:
#Title    
    title = i.find('h3').text
#Full res link
    baseImageUrl = i.find('a', class_ = 'itemLink product-item')['href']
#Main link
    hemiMainUrl = 'https://astrogeology.usgs.gov'
#Visit full link
    browser.visit(hemiMainUrl + baseImageUrl)
#Another html object
    hemiImageHtml = browser.html

#Ramen time
    soup = bs(hemiImageHtml, 'html.parser')
    
#Full size image source
    imageUrl = hemiMainUrl + soup.find('img', class_ = 'wide-image')['src']
    
#Lets put it all together now
    hemiImages.append({'title': title, 'imageUrl' : imageUrl})
    #hemiTitles.append({'title': title})
    #print(f'{title}')
    print(f'{imageUrl}')
    

https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg
https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg
https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg
https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg


In [10]:
hemiImages


[{'title': 'Cerberus Hemisphere Enhanced',
  'imageUrl': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'imageUrl': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'imageUrl': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'imageUrl': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'}]

Convert notebook to python script

In [ ]:
#!jupyter nbconvert --to python mission_to_mars.ipynb